In [17]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np
import random

# 시드 및 모델의 기본적인 파라미터 고정

In [18]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

batch_size = 32
num_epochs = 200
learning_rate = 0.0001

# GPU 확인 및 device에 사용할 gpu 저장

In [19]:
print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.7.1
GPU 사용 가능 여부: True


# 데이터셋 만들기

In [20]:
class SeperateDataset(Dataset): # 각 목표별 데이터셋 만들기
    def __init__(self, df, transform, target):
        self.df = df
        self.img_paths = self.df['path'].tolist()
        self.labels = self.df[target].tolist()
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(self.labels[index])

    def __len__(self):
        return len(self.img_paths)
    
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

# transform 만들기
### 아래 부분은 3x384x384 할지 3x224x224할지 조건에 따라 선택해서 사용하시면 됩니다.
### 각각 resize 후에 80퍼센트만 남도록 crop을 하였습니다.

In [21]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((280,280)),
        transforms.CenterCrop(224),
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((280,280)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}



# 아래 영역을 마우스로 지정하고, 컨트롤키 + '/' 눌러서 주석 제거
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.Resize((480,480)),
#         transforms.CenterCrop(384),
#         transforms.RandomRotation(5),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'test': transforms.Compose([
#         transforms.Resize((480,480)),
#         transforms.CenterCrop(384),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }

# 학습할 데이터 불러오기
### 이 아래 부분은 실험 조건에 따라서 주석처리하면서 돌리면 됩니다.

In [22]:
# 5:1:1
#df = pd.read_csv("/opt/ml/code/total.csv")

# 1:1:1
df = pd.read_csv("/opt/ml/code/total_111.csv", index_col=None)
list_columns = list(df.columns)

In [24]:
df

,gender,age,maskOX,maskGB,class,path
0,1,1,0,NaN,4,/opt/ml/input/data/train/images/000001_female_...
1,1,1,1,0.0,16,/opt/ml/input/data/train/images/000001_female_...
2,1,1,1,1.0,10,/opt/ml/input/data/train/images/000001_female_...
3,1,1,0,NaN,4,/opt/ml/input/data/train/images/000002_female_...
4,1,1,1,0.0,16,/opt/ml/input/data/train/images/000002_female_...
...,...,...,...,...,...,...
8095,0,0,1,0.0,12,/opt/ml/input/data/train/images/006957_male_As...
8096,0,0,1,1.0,6,/opt/ml/input/data/train/images/006957_male_As...
8097,0,0,0,NaN,0,/opt/ml/input/data/train/images/006959_male_As...
8098,0,0,1,0.0,12,/opt/ml/input/data/train/images/006959_male_As...


In [25]:
test_dir = '/opt/ml/input/data/eval/'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

### 만들 타겟을 아래에 적어주세요 (gender/age/maskOX/maskGB/class)

In [26]:
target ="class"
list_columns.remove(target)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[list_columns], df[target], test_size=0.2, stratify=df[target], random_state=42)

In [28]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [29]:
train_data = SeperateDataset(train_df, transform=data_transforms['train'], target=target)
val_data = SeperateDataset(test_df, transform=data_transforms['test'], target=target)
test_data = TestDataset(image_paths, transform=data_transforms['test'])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [30]:
import timeit

def train(num_epochs, model, data_loader, criterion, optimizer, saved_dir, val_every, device):
    print('Start training..')
    total_start_time = timeit.default_timer()
    best_loss = 9999999
    best_test_accuracy = 0
    for epoch in tqdm(range(num_epochs)):
        epoch_f1 = 0
        running_acc = 0
        print('Epoch start..')
        epoch_start_time = timeit.default_timer()
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()         
            loss.backward()
            optimizer.step()

            _, argmax = torch.max(outputs, 1)
            accuracy = (labels == argmax).float().mean()
            
            f1 = f1_score(labels.cpu().numpy(), argmax.cpu().numpy(), average='macro')
            epoch_f1 += f1
            running_acc += accuracy
            if (i+1) % 3 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, F1_Score: {:.2f}'.format(
                    epoch+1, num_epochs, i+1, len(data_loader), loss.item(), accuracy.item() * 100, f1))
        print("------------Epoch Finish------------")
        print('Epoch [{}/{}], Accuracy: {:.2f}%, F1_Score: {:.2f}'.format(
                    epoch+1, num_epochs, running_acc.item()/(i+1) * 100,epoch_f1/(i+1)))
        if (epoch + 1) % val_every == 0:
            avrg_loss = validation(epoch + 1, model, val_loader, criterion, device)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, saved_dir)
        epoch_end_time = timeit.default_timer()
        print("Epoch end..")
        print(f"epoch time : {epoch_end_time-epoch_start_time}")
        epoch_acc = running_acc / (i+1)
        
        if best_test_accuracy < epoch_acc:
            best_test_accuracy = epoch_acc
            save_model(model, saved_dir)
            early_stop_point = 0
        else:
            early_stop_point += 1
        if early_stop_point == 3:
            print('early_stopped')
            break
    print('End training..')
    total_end_time = timeit.default_timer()
    print(f"total time : {total_end_time-total_start_time}")

In [31]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch) )
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_loss = 0
        cnt = 0
        epoch_f1 = 0
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            ## 코드 종료 ##
            total += imgs.size(0)
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item()
            total_loss += loss
            cnt += 1
            epoch_f1 += f1_score(labels.cpu().numpy(), argmax.cpu().numpy(), average='macro')
        avrg_loss = total_loss / cnt
        avrg_f1 = epoch_f1 / cnt
        print('Validation #{}  Accuracy: {:.2f}% F1_Score: {:.2f} Average Loss: {:.4f}'.format(epoch, correct / total * 100,avrg_f1 ,avrg_loss))
    model.train()
    return avrg_loss

In [32]:
def save_model(model, saved_dir, file_name='best_model.pt'):
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    torch.save(check_point,output_path)

In [33]:
# pip install timm

In [34]:
import timm

class ViTBase32_224(nn.Module):
    def __init__(self, n_classes):

        super(ViTBase32_224, self).__init__()

        self.model = timm.create_model("vit_base_patch32_224", pretrained=True)
        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return x

In [35]:
# target에 따라서 n_classes 정하면 된다.
model = ViTBase32_224(n_classes=18)

In [36]:
# 끝에서 학습할 레이어의 수를 설정합니다.
num_train_layer = 1

for i, param in enumerate(model.parameters()):
    if i == len(list(model.parameters())) - num_train_layer:
        break
    param.requires_grad = False

In [37]:
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

saved_dir = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/model'
val_every = 1

In [38]:
from torchsummary import summary as summary_

summary_(model,(3,224,224),batch_size=32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [32, 768, 7, 7]       2,360,064
          Identity-2              [32, 49, 768]               0
        PatchEmbed-3              [32, 49, 768]               0
           Dropout-4              [32, 50, 768]               0
         LayerNorm-5              [32, 50, 768]           1,536
            Linear-6             [32, 50, 2304]       1,771,776
           Dropout-7           [32, 12, 50, 50]               0
            Linear-8              [32, 50, 768]         590,592
           Dropout-9              [32, 50, 768]               0
        Attention-10              [32, 50, 768]               0
         Identity-11              [32, 50, 768]               0
        LayerNorm-12              [32, 50, 768]           1,536
           Linear-13             [32, 50, 3072]       2,362,368
             GELU-14             [32, 5

In [ ]:
train(num_epochs, model, train_loader, criterion, optimizer, saved_dir, val_every, device)

  0%|          | 0/200 [00:00<?, ?it/s]

Start training..
Epoch start..
Epoch [1/200], Step [3/202], Loss: 0.7167, Accuracy: 62.50%, F1_Score: 0.61
Epoch [1/200], Step [6/202], Loss: 0.6050, Accuracy: 59.38%, F1_Score: 0.59
Epoch [1/200], Step [9/202], Loss: 0.5435, Accuracy: 78.12%, F1_Score: 0.77
Epoch [1/200], Step [12/202], Loss: 0.6677, Accuracy: 62.50%, F1_Score: 0.62
Epoch [1/200], Step [15/202], Loss: 0.7657, Accuracy: 59.38%, F1_Score: 0.59
Epoch [1/200], Step [18/202], Loss: 0.7444, Accuracy: 56.25%, F1_Score: 0.56
Epoch [1/200], Step [21/202], Loss: 0.6687, Accuracy: 65.62%, F1_Score: 0.65
Epoch [1/200], Step [24/202], Loss: 0.7123, Accuracy: 59.38%, F1_Score: 0.59
Epoch [1/200], Step [27/202], Loss: 0.6969, Accuracy: 59.38%, F1_Score: 0.59
Epoch [1/200], Step [30/202], Loss: 0.8209, Accuracy: 50.00%, F1_Score: 0.50
Epoch [1/200], Step [33/202], Loss: 0.9141, Accuracy: 50.00%, F1_Score: 0.45
Epoch [1/200], Step [36/202], Loss: 0.7782, Accuracy: 53.12%, F1_Score: 0.49
Epoch [1/200], Step [39/202], Loss: 0.7227, Accu

  0%|          | 1/200 [01:03<3:31:30, 63.77s/it]

Epoch start..
Epoch [2/200], Step [3/202], Loss: 0.6868, Accuracy: 62.50%, F1_Score: 0.62
Epoch [2/200], Step [6/202], Loss: 0.5950, Accuracy: 65.62%, F1_Score: 0.64
Epoch [2/200], Step [9/202], Loss: 0.6895, Accuracy: 65.62%, F1_Score: 0.63
Epoch [2/200], Step [12/202], Loss: 0.6797, Accuracy: 62.50%, F1_Score: 0.62
Epoch [2/200], Step [15/202], Loss: 0.7317, Accuracy: 53.12%, F1_Score: 0.53
Epoch [2/200], Step [18/202], Loss: 0.6023, Accuracy: 71.88%, F1_Score: 0.72
Epoch [2/200], Step [21/202], Loss: 0.8645, Accuracy: 50.00%, F1_Score: 0.50
Epoch [2/200], Step [24/202], Loss: 0.6588, Accuracy: 62.50%, F1_Score: 0.61
Epoch [2/200], Step [27/202], Loss: 0.6368, Accuracy: 59.38%, F1_Score: 0.59
Epoch [2/200], Step [30/202], Loss: 0.5190, Accuracy: 81.25%, F1_Score: 0.80
Epoch [2/200], Step [33/202], Loss: 0.7417, Accuracy: 56.25%, F1_Score: 0.56
Epoch [2/200], Step [36/202], Loss: 0.6574, Accuracy: 65.62%, F1_Score: 0.65
Epoch [2/200], Step [39/202], Loss: 0.6730, Accuracy: 59.38%, F1_

  1%|          | 2/200 [02:07<3:30:26, 63.77s/it]

...

Epoch start..
Epoch [5/200], Step [3/202], Loss: 0.5457, Accuracy: 81.25%, F1_Score: 0.81
Epoch [5/200], Step [6/202], Loss: 0.5884, Accuracy: 59.38%, F1_Score: 0.59
Epoch [5/200], Step [9/202], Loss: 0.7868, Accuracy: 53.12%, F1_Score: 0.52
Epoch [5/200], Step [12/202], Loss: 0.7887, Accuracy: 59.38%, F1_Score: 0.59
Epoch [5/200], Step [15/202], Loss: 0.5705, Accuracy: 68.75%, F1_Score: 0.68
Epoch [5/200], Step [18/202], Loss: 0.7993, Accuracy: 53.12%, F1_Score: 0.53
Epoch [5/200], Step [21/202], Loss: 0.7230, Accuracy: 56.25%, F1_Score: 0.56
Epoch [5/200], Step [24/202], Loss: 0.6819, Accuracy: 65.62%, F1_Score: 0.66
Epoch [5/200], Step [27/202], Loss: 0.6969, Accuracy: 65.62%, F1_Score: 0.64
Epoch [5/200], Step [30/202], Loss: 0.6679, Accuracy: 59.38%, F1_Score: 0.56
Epoch [5/200], Step [33/202], Loss: 0.7379, Accuracy: 59.38%, F1_Score: 0.56
Epoch [5/200], Step [36/202], Loss: 0.8069, Accuracy: 53.12%, F1_Score: 0.53
Epoch [5/200], Step [39/202], Loss: 0.6551, Accuracy: 59.38%, F1_

  2%|▎         | 5/200 [05:18<3:27:16, 63.77s/it]

Epoch start..
Epoch [6/200], Step [3/202], Loss: 0.5979, Accuracy: 65.62%, F1_Score: 0.66
Epoch [6/200], Step [6/202], Loss: 0.6298, Accuracy: 71.88%, F1_Score: 0.70
Epoch [6/200], Step [9/202], Loss: 0.6605, Accuracy: 65.62%, F1_Score: 0.65
Epoch [6/200], Step [12/202], Loss: 0.7307, Accuracy: 46.88%, F1_Score: 0.42
Epoch [6/200], Step [15/202], Loss: 0.6502, Accuracy: 68.75%, F1_Score: 0.69
Epoch [6/200], Step [18/202], Loss: 0.5016, Accuracy: 78.12%, F1_Score: 0.78
Epoch [6/200], Step [21/202], Loss: 0.6695, Accuracy: 68.75%, F1_Score: 0.69
Epoch [6/200], Step [24/202], Loss: 0.7426, Accuracy: 62.50%, F1_Score: 0.62
Epoch [6/200], Step [27/202], Loss: 0.8966, Accuracy: 43.75%, F1_Score: 0.43
Epoch [6/200], Step [30/202], Loss: 0.7206, Accuracy: 50.00%, F1_Score: 0.50
Epoch [6/200], Step [33/202], Loss: 0.7231, Accuracy: 56.25%, F1_Score: 0.56
Epoch [6/200], Step [36/202], Loss: 0.6414, Accuracy: 56.25%, F1_Score: 0.52
Epoch [6/200], Step [39/202], Loss: 0.7074, Accuracy: 53.12%, F1_

  3%|▎         | 6/200 [06:22<3:26:11, 63.77s/it]

Epoch start..
Epoch [7/200], Step [3/202], Loss: 0.6753, Accuracy: 65.62%, F1_Score: 0.65
Epoch [7/200], Step [6/202], Loss: 0.6946, Accuracy: 53.12%, F1_Score: 0.53
Epoch [7/200], Step [9/202], Loss: 0.6062, Accuracy: 68.75%, F1_Score: 0.69
Epoch [7/200], Step [12/202], Loss: 0.6932, Accuracy: 62.50%, F1_Score: 0.62
Epoch [7/200], Step [15/202], Loss: 0.7053, Accuracy: 59.38%, F1_Score: 0.59
Epoch [7/200], Step [18/202], Loss: 0.6083, Accuracy: 68.75%, F1_Score: 0.69
Epoch [7/200], Step [21/202], Loss: 0.9444, Accuracy: 50.00%, F1_Score: 0.50
Epoch [7/200], Step [24/202], Loss: 0.6710, Accuracy: 59.38%, F1_Score: 0.57
Epoch [7/200], Step [27/202], Loss: 0.6694, Accuracy: 59.38%, F1_Score: 0.59
Epoch [7/200], Step [30/202], Loss: 0.7086, Accuracy: 59.38%, F1_Score: 0.58
Epoch [7/200], Step [33/202], Loss: 0.8056, Accuracy: 53.12%, F1_Score: 0.53
Epoch [7/200], Step [36/202], Loss: 0.6782, Accuracy: 59.38%, F1_Score: 0.59
Epoch [7/200], Step [39/202], Loss: 0.6728, Accuracy: 65.62%, F1_

In [ ]:
model_path = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/model/best_model.pt'
checkpoint = torch.load(model_path,map_location=device)
state_dict = checkpoint['net']
model.load_state_dict(state_dict)

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submit_dir = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/submit'
submission.to_csv(os.path.join(submit_dir, 'submission.csv'), index=False)
print('test inference is done!')